# パワーヒッター vs アベレージヒッター
## 目的
1. どちらがOPSが高いのか（試合貢献度）
2. どちらが年俸が高いのか
3. 年齢や血液型との関係はあるのか（２つのグループ間で）

## 分類
- パワーヒッター: 三振率≥ 75%タイル（約0.23）  → 三振が多い AND HR≥15本 AND 打率<.280 AND 試合≥100
- アベレージヒッター: 三振率≤ 25%タイル（約0.15）→ 三振が少ない AND 打率≥.280 AND HR<15本 AND 試合≥100

### 1. ライブラリのインポート

In [ ]:
# 一度pip install したらコメントアウトしてください！！
# !pip install japanize-matplotlib
# !pip install tabulate
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import os
from scipy import stats
from math import pi

# グラフのスタイル設定
plt.rcParams['font.family'] = 'Hiragino Maru Gothic Pro'
plt.rcParams['axes.unicode_minus'] = False
sns.set_palette("husl")

print("✓ ライブラリのインポート完了")

### 2. データの読み込みと結合

In [ ]:
# stats_bat.csv の読み込み
path_pattern_1 = 'work_dataset/stats_bat.csv'
path_pattern_2 = 'stats_bat.csv'
use_file_path = path_pattern_1 if os.path.exists(path_pattern_1) else path_pattern_2

try:
    df = pd.read_csv(use_file_path, encoding='Shift-JIS')
    print("✓ Shift-JISでstats_bat.csvを読み込みました")
except UnicodeDecodeError:
    df = pd.read_csv(use_file_path, encoding='cp932')
    print("✓ cp932でstats_bat.csvを読み込みました")

# players.csv の読み込み（年俸・年齢・血液型データ）
players_path_1 = 'work_dataset/players.csv'
players_path_2 = 'players.csv'
players_path = players_path_1 if os.path.exists(players_path_1) else players_path_2

try:
    players = pd.read_csv(players_path, encoding='Shift-JIS')
    print("✓ Shift-JISでplayers.csvを読み込みました")
except UnicodeDecodeError:
    players = pd.read_csv(players_path, encoding='cp932')
    print("✓ cp932でplayers.csvを読み込みました")

print(f"\nstats_bat.csv: {len(df)}行")
print(f"players.csv: {len(players)}行")
print("-" * 50)

### 3. 打撃指標の計算

In [ ]:
# 必要なカラムを選択
columns_to_keep = ['player_id', 'player_name', 'season_year', 'pa', 'ab', 'h', 'tb', 'bb', 'hbp', 'sf', 'so', 'hr', 'g']
df_bat = df[columns_to_keep].copy()

# 規定打席数（100打席以上）
MIN_PA = 100
df_bat = df_bat[df_bat['pa'] >= MIN_PA].reset_index(drop=True)

# 打率 (BA)
df_bat['BA'] = np.where(df_bat['ab'] > 0, df_bat['h'] / df_bat['ab'], np.nan)

# 出塁率 (OBP)
df_bat['OBP_denominator'] = df_bat['ab'] + df_bat['bb'] + df_bat['hbp'] + df_bat['sf']
df_bat['OBP_numerator'] = df_bat['h'] + df_bat['bb'] + df_bat['hbp']
df_bat['OBP'] = np.where(df_bat['OBP_denominator'] > 0, 
                         df_bat['OBP_numerator'] / df_bat['OBP_denominator'], np.nan)

# 長打率 (SLG)
df_bat['SLG'] = np.where(df_bat['ab'] > 0, df_bat['tb'] / df_bat['ab'], np.nan)

# OPS
df_bat['OPS'] = df_bat['OBP'] + df_bat['SLG']

# 三振率 (K%)
df_bat['K_Rate'] = df_bat['so'] / df_bat['pa']

# ホームラン率 (HR%)
df_bat['HR_Rate'] = df_bat['hr'] / df_bat['pa']

# 欠損値を除外
df_bat.dropna(subset=['OPS', 'K_Rate', 'HR_Rate', 'BA'], inplace=True)

print(f"✓ 規定打席以上の選手: {len(df_bat)}名")
print("-" * 50)

### 4. 選手のカテゴリ分類

In [ ]:
# 三振率のパーセンタイル
k_rate_q1 = df_bat['K_Rate'].quantile(0.25)  # 下位25%
k_rate_q3 = df_bat['K_Rate'].quantile(0.75)  # 上位25%

print(f"三振率 25パーセンタイル: {k_rate_q1:.3f} ({k_rate_q1*100:.1f}%)")
print(f"三振率 75パーセンタイル: {k_rate_q3:.3f} ({k_rate_q3*100:.1f}%)")
print("-" * 50)

# カテゴリ分類の関数（相互排他的）
def categorize_player(row):
    # パワーヒッター: 三振率高 + ホームラン多 + 打率低
    if row['K_Rate'] >= k_rate_q3 and row['hr'] >= 15 and row['BA'] < 0.280 and row['g'] >= 100:
        return 'パワーヒッター'
    # アベレージヒッター: 三振率低 + 打率高 + ホームラン少
    elif row['K_Rate'] <= k_rate_q1 and row['BA'] >= 0.280 and row['hr'] < 15 and row['g'] >= 100:
        return 'アベレージヒッター'
    else:
        return 'その他'

df_bat['category'] = df_bat.apply(categorize_player, axis=1)

# カテゴリ別の選手数
category_counts = df_bat['category'].value_counts()
print("【カテゴリ別選手数】")
print(category_counts)
print("-" * 50)

# 重複チェック
overlap = df_bat[(df_bat['category'] == 'パワーヒッター') | (df_bat['category'] == 'アベレージヒッター')]
if overlap[['hr','BA','K_Rate']].duplicated().any():
    print("⚠️ 警告: 重複している選手がいます！")
else:
    print("✓ 重複なし: 各選手は1つのカテゴリにのみ分類されています")

### 5. 年齢・血液型・年俸データとの結合

In [ ]:
# 誕生日データを日時型に変換
players['birthday_date'] = pd.to_datetime(players['birthday_date'], errors='coerce')

# player_idで結合
df_merged = pd.merge(df_bat, players[['player_id', 'salary', 'blood_type', 'birthday_date']], 
                     on='player_id', how='left')

# シーズン年齢を計算
def calc_age(row):
    if pd.isnull(row['birthday_date']):
        return None
    return row['season_year'] - row['birthday_date'].year

df_merged['age'] = df_merged.apply(calc_age, axis=1)

# データがある選手のみ（年俸・年齢・血液型）
df_complete = df_merged.dropna(subset=['salary', 'age', 'blood_type']).copy()

# カテゴリ別に抽出
df_power = df_complete[df_complete['category'] == 'パワーヒッター'].copy()
df_average = df_complete[df_complete['category'] == 'アベレージヒッター'].copy()

print(f"✓ 完全データあり - パワーヒッター: {len(df_power)}名")
print(f"✓ 完全データあり - アベレージヒッター: {len(df_average)}名")
print("-" * 50)

# 分析1: OPS比較（試合貢献度）

In [ ]:
print("="*100)
print("【分析1: OPS比較（試合貢献度）】")
print("="*100)

# 基本統計量
print("\n【基本統計量】")
print(f"パワーヒッター:")
print(f"  平均OPS: {df_power['OPS'].mean():.3f}")
print(f"  中央値: {df_power['OPS'].median():.3f}")
print(f"  標準偏差: {df_power['OPS'].std():.3f}")
print(f"  最高: {df_power['OPS'].max():.3f}")
print(f"  最低: {df_power['OPS'].min():.3f}")

print(f"\nアベレージヒッター:")
print(f"  平均OPS: {df_average['OPS'].mean():.3f}")
print(f"  中央値: {df_average['OPS'].median():.3f}")
print(f"  標準偏差: {df_average['OPS'].std():.3f}")
print(f"  最高: {df_average['OPS'].max():.3f}")
print(f"  最低: {df_average['OPS'].min():.3f}")

# t検定
t_stat_ops, p_value_ops = stats.ttest_ind(df_power['OPS'], df_average['OPS'])

print(f"\n【統計的検定】")
print(f"t統計量: {t_stat_ops:.4f}")
print(f"p値: {p_value_ops:.4f}")

if p_value_ops < 0.05:
    print("\n結論: 統計的に有意な差あり（5%水準）")
    if df_power['OPS'].mean() > df_average['OPS'].mean():
        ops_winner = "パワーヒッター"
        print(f"→ {ops_winner}の方がOPSが高い")
    else:
        ops_winner = "アベレージヒッター"
        print(f"→ {ops_winner}の方がOPSが高い")
else:
    ops_winner = "同程度"
    print("\n結論: 統計的に有意な差なし")
    print("→ どちらも同程度の試合貢献度")

print("-" * 100)

# 分析2: 年俸比較（市場価値）

In [ ]:
print("="*100)
print("【分析2: 年俸比較（市場価値）】")
print("="*100)

# 基本統計量
print("\n【基本統計量】")
print(f"パワーヒッター:")
print(f"  平均年俸: {df_power['salary'].mean():,.0f}万円 ")
print(f"  中央値: {df_power['salary'].median():,.0f}万円 ")
print(f"  標準偏差: {df_power['salary'].std():,.0f}万円")
print(f"  最高年俸: {df_power['salary'].max():,.0f}万円 ")
print(f"  最低年俸: {df_power['salary'].min():,.0f}万円 ")

print(f"\nアベレージヒッター:")
print(f"  平均年俸: {df_average['salary'].mean():,.0f}万円 ")
print(f"  中央値: {df_average['salary'].median():,.0f}万円 ")
print(f"  標準偏差: {df_average['salary'].std():,.0f}万円")
print(f"  最高年俸: {df_average['salary'].max():,.0f}万円 ")
print(f"  最低年俸: {df_average['salary'].min():,.0f}万円 ")

# t検定
t_stat_salary, p_value_salary = stats.ttest_ind(df_power['salary'], df_average['salary'])

print(f"\n【統計的検定】")
print(f"t統計量: {t_stat_salary:.4f}")
print(f"p値: {p_value_salary:.4f}")

if p_value_salary < 0.05:
    print("\n結論: 統計的に有意な差あり（5%水準）")
    if df_power['salary'].mean() > df_average['salary'].mean():
        salary_winner = "パワーヒッター"
        print(f"→ {salary_winner}の方が年俸が高い")
    else:
        salary_winner = "アベレージヒッター"
        print(f"→ {salary_winner}の方が年俸が高い")
else:
    salary_winner = "同程度"
    print("\n結論: 統計的に有意な差なし")
    print("→ どちらも同程度の市場価値")

# コストパフォーマンス
df_power['OPS_per_10M'] = df_power['OPS'] / (df_power['salary'] / 1000)
df_average['OPS_per_10M'] = df_average['OPS'] / (df_average['salary'] / 1000)

print(f"\n【コストパフォーマンス（OPS per 1000万円）】")
print(f"パワーヒッター: {df_power['OPS_per_10M'].mean():.4f}")
print(f"アベレージヒッター: {df_average['OPS_per_10M'].mean():.4f}")

if df_power['OPS_per_10M'].mean() > df_average['OPS_per_10M'].mean():
    cp_winner = "パワーヒッター"
else:
    cp_winner = "アベレージヒッター"

print(f"→ {cp_winner}の方がコストパフォーマンスが高い")
print("-" * 100)

# 分析3: 年齢との関係

In [ ]:
print("="*100)
print("【分析3: 年齢との関係】")
print("="*100)

# 年齢の基本統計量
print("\n【年齢の基本統計量】")
print(f"パワーヒッター:")
print(f"  平均年齢: {df_power['age'].mean():.1f}歳")
print(f"  中央値: {df_power['age'].median():.1f}歳")
print(f"  標準偏差: {df_power['age'].std():.1f}歳")
print(f"  最高齢: {df_power['age'].max():.0f}歳")
print(f"  最年少: {df_power['age'].min():.0f}歳")

print(f"\nアベレージヒッター:")
print(f"  平均年齢: {df_average['age'].mean():.1f}歳")
print(f"  中央値: {df_average['age'].median():.1f}歳")
print(f"  標準偏差: {df_average['age'].std():.1f}歳")
print(f"  最高齢: {df_average['age'].max():.0f}歳")
print(f"  最年少: {df_average['age'].min():.0f}歳")

# 年齢のt検定
t_stat_age, p_value_age = stats.ttest_ind(df_power['age'], df_average['age'])

print(f"\n【年齢の統計的検定】")
print(f"t統計量: {t_stat_age:.4f}")
print(f"p値: {p_value_age:.4f}")

if p_value_age < 0.05:
    print("\n結論: 年齢に統計的に有意な差あり")
    if df_power['age'].mean() > df_average['age'].mean():
        print("→ パワーヒッターの方が年齢が高い傾向")
    else:
        print("→ アベレージヒッターの方が年齢が高い傾向")
else:
    print("\n結論: 年齢に統計的に有意な差なし")
    print("→ 打撃スタイルと年齢に明確な関係はない")

# 年齢とOPSの相関
print(f"\n【年齢とOPSの相関】")
corr_age_ops_power = df_power[['age', 'OPS']].corr().iloc[0, 1]
corr_age_ops_average = df_average[['age', 'OPS']].corr().iloc[0, 1]

print(f"パワーヒッター: {corr_age_ops_power:.3f}")
print(f"アベレージヒッター: {corr_age_ops_average:.3f}")

# 年齢と年俸の相関
print(f"\n【年齢と年俸の相関】")
corr_age_salary_power = df_power[['age', 'salary']].corr().iloc[0, 1]
corr_age_salary_average = df_average[['age', 'salary']].corr().iloc[0, 1]

print(f"パワーヒッター: {corr_age_salary_power:.3f}")
print(f"アベレージヒッター: {corr_age_salary_average:.3f}")

print("-" * 100)

# 分析4: 血液型との関係

In [ ]:
print("="*100)
print("【分析4: 血液型との関係】")
print("="*100)

# 血液型の分布
print("\n【血液型の分布】")
print("\nパワーヒッター:")
blood_power = df_power['blood_type'].value_counts()
for bt, count in blood_power.items():
    pct = (count / len(df_power)) * 100
    print(f"  {bt}型: {count}名 ({pct:.1f}%)")

print("\nアベレージヒッター:")
blood_average = df_average['blood_type'].value_counts()
for bt, count in blood_average.items():
    pct = (count / len(df_average)) * 100
    print(f"  {bt}型: {count}名 ({pct:.1f}%)")

# 血液型別のOPS平均
print(f"\n【血液型別のOPS平均】")
print("\nパワーヒッター:")
ops_by_blood_power = df_power.groupby('blood_type')['OPS'].mean().sort_values(ascending=False)
for bt, ops in ops_by_blood_power.items():
    print(f"  {bt}型: {ops:.3f}")

print("\nアベレージヒッター:")
ops_by_blood_average = df_average.groupby('blood_type')['OPS'].mean().sort_values(ascending=False)
for bt, ops in ops_by_blood_average.items():
    print(f"  {bt}型: {ops:.3f}")

# 血液型別の年俸平均
print(f"\n【血液型別の年俸平均】")
print("\nパワーヒッター:")
salary_by_blood_power = df_power.groupby('blood_type')['salary'].mean().sort_values(ascending=False)
for bt, salary in salary_by_blood_power.items():
    print(f"  {bt}型: {salary:.0f}万円")

print("\nアベレージヒッター:")
salary_by_blood_average = df_average.groupby('blood_type')['salary'].mean().sort_values(ascending=False)
for bt, salary in salary_by_blood_average.items():
    print(f"  {bt}型: {salary:.0f}万円")

# カイ二乗検定（血液型の分布に差があるか）
print(f"\n【カイ二乗検定：血液型分布の差】")
# 両グループで共通の血液型のみを使用
common_blood_types = set(df_power['blood_type'].unique()) & set(df_average['blood_type'].unique())
if len(common_blood_types) >= 2:
    contingency_table = pd.crosstab(df_complete[df_complete['category'].isin(['パワーヒッター', 'アベレージヒッター'])]['category'],
                                    df_complete[df_complete['category'].isin(['パワーヒッター', 'アベレージヒッター'])]['blood_type'])
    chi2, p_value_blood, dof, expected = stats.chi2_contingency(contingency_table)
    print(f"カイ二乗統計量: {chi2:.4f}")
    print(f"p値: {p_value_blood:.4f}")
    
    if p_value_blood < 0.05:
        print("\n結論: 血液型の分布に統計的に有意な差あり")
        print("→ 打撃スタイルと血液型に関係がある可能性")
    else:
        print("\n結論: 血液型の分布に統計的に有意な差なし")
        print("→ 打撃スタイルと血液型に明確な関係はない")
else:
    print("血液型データが不足しているため、カイ二乗検定をスキップしました")

print("-" * 100)

# 可視化

### 可視化1: OPS・年俸・年齢の比較

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

df_viz = pd.concat([df_power, df_average])

# OPS比較
ax1 = axes[0]
sns.boxplot(data=df_viz, x='category', y='OPS', ax=ax1, palette=['#FF4444', '#4444FF'])
ax1.set_title('OPS比較', fontsize=16, fontweight='bold')
ax1.set_xlabel('')
ax1.set_ylabel('OPS', fontsize=13)
ax1.grid(True, alpha=0.3, axis='y')

# 年俸比較
ax2 = axes[1]
sns.boxplot(data=df_viz, x='category', y='salary', ax=ax2, palette=['#FF4444', '#4444FF'])
ax2.set_title('年俸比較', fontsize=16, fontweight='bold')
ax2.set_xlabel('')
ax2.set_ylabel('年俸（万円）', fontsize=13)
ax2.ticklabel_format(style='plain', axis='y')
ax2.grid(True, alpha=0.3, axis='y')

# 年齢比較
ax3 = axes[2]
sns.boxplot(data=df_viz, x='category', y='age', ax=ax3, palette=['#FF4444', '#4444FF'])
ax3.set_title('年齢比較', fontsize=16, fontweight='bold')
ax3.set_xlabel('')
ax3.set_ylabel('年齢（歳）', fontsize=13)
ax3.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('basic_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ グラフを 'basic_comparison.png' として保存しました。")

### 可視化2: 年齢とOPS・年俸の関係

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 年齢 vs OPS
ax1 = axes[0]
ax1.scatter(df_power['age'], df_power['OPS'], alpha=0.7, color='#FF4444', s=100, 
           edgecolors='white', linewidth=1.5, label='パワーヒッター')
ax1.scatter(df_average['age'], df_average['OPS'], alpha=0.7, color='#4444FF', s=100,
           edgecolors='white', linewidth=1.5, label='アベレージヒッター')

# 回帰線
z_power = np.polyfit(df_power['age'], df_power['OPS'], 1)
p_power = np.poly1d(z_power)
ax1.plot(df_power['age'].sort_values(), p_power(df_power['age'].sort_values()), 
        "r--", alpha=0.8, linewidth=2, label=f'パワー回帰線 (r={corr_age_ops_power:.3f})')

z_average = np.polyfit(df_average['age'], df_average['OPS'], 1)
p_average = np.poly1d(z_average)
ax1.plot(df_average['age'].sort_values(), p_average(df_average['age'].sort_values()), 
        "b--", alpha=0.8, linewidth=2, label=f'アベレージ回帰線 (r={corr_age_ops_average:.3f})')

ax1.set_title('年齢とOPSの関係', fontsize=16, fontweight='bold')
ax1.set_xlabel('年齢（歳）', fontsize=13)
ax1.set_ylabel('OPS', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 年齢 vs 年俸
ax2 = axes[1]
ax2.scatter(df_power['age'], df_power['salary'], alpha=0.7, color='#FF4444', s=100,
           edgecolors='white', linewidth=1.5, label='パワーヒッター')
ax2.scatter(df_average['age'], df_average['salary'], alpha=0.7, color='#4444FF', s=100,
           edgecolors='white', linewidth=1.5, label='アベレージヒッター')

# 回帰線
z_power_s = np.polyfit(df_power['age'], df_power['salary'], 1)
p_power_s = np.poly1d(z_power_s)
ax2.plot(df_power['age'].sort_values(), p_power_s(df_power['age'].sort_values()), 
        "r--", alpha=0.8, linewidth=2, label=f'パワー回帰線 (r={corr_age_salary_power:.3f})')

z_average_s = np.polyfit(df_average['age'], df_average['salary'], 1)
p_average_s = np.poly1d(z_average_s)
ax2.plot(df_average['age'].sort_values(), p_average_s(df_average['age'].sort_values()), 
        "b--", alpha=0.8, linewidth=2, label=f'アベレージ回帰線 (r={corr_age_salary_average:.3f})')

ax2.set_title('年齢と年俸の関係', fontsize=16, fontweight='bold')
ax2.set_xlabel('年齢（歳）', fontsize=13)
ax2.set_ylabel('年俸（万円）', fontsize=13)
ax2.ticklabel_format(style='plain', axis='y')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('age_relationship.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ グラフを 'age_relationship.png' として保存しました。")

### 可視化3: 血液型別の比較

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 血液型分布（パワーヒッター）
ax1 = axes[0, 0]
blood_power.plot(kind='bar', ax=ax1, color='#FF4444', alpha=0.7, edgecolor='black')
ax1.set_title('血液型分布（パワーヒッター）', fontsize=14, fontweight='bold')
ax1.set_xlabel('血液型', fontsize=12)
ax1.set_ylabel('選手数', fontsize=12)
ax1.tick_params(axis='x', rotation=0)
ax1.grid(True, alpha=0.3, axis='y')

# 血液型分布（アベレージヒッター）
ax2 = axes[0, 1]
blood_average.plot(kind='bar', ax=ax2, color='#4444FF', alpha=0.7, edgecolor='black')
ax2.set_title('血液型分布（アベレージヒッター）', fontsize=14, fontweight='bold')
ax2.set_xlabel('血液型', fontsize=12)
ax2.set_ylabel('選手数', fontsize=12)
ax2.tick_params(axis='x', rotation=0)
ax2.grid(True, alpha=0.3, axis='y')

# 血液型別OPS（パワーヒッター）
ax3 = axes[1, 0]
ops_by_blood_power.plot(kind='bar', ax=ax3, color='#FF4444', alpha=0.7, edgecolor='black')
ax3.set_title('血液型別OPS（パワーヒッター）', fontsize=14, fontweight='bold')
ax3.set_xlabel('血液型', fontsize=12)
ax3.set_ylabel('平均OPS', fontsize=12)
ax3.tick_params(axis='x', rotation=0)
ax3.grid(True, alpha=0.3, axis='y')

# 血液型別OPS（アベレージヒッター）
ax4 = axes[1, 1]
ops_by_blood_average.plot(kind='bar', ax=ax4, color='#4444FF', alpha=0.7, edgecolor='black')
ax4.set_title('血液型別OPS（アベレージヒッター）', fontsize=14, fontweight='bold')
ax4.set_xlabel('血液型', fontsize=12)
ax4.set_ylabel('平均OPS', fontsize=12)
ax4.tick_params(axis='x', rotation=0)
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('blood_type_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ グラフを 'blood_type_analysis.png' として保存しました。")

# 総合的なまとめ

In [ ]:
print("【総合分析結果まとめ】")

print("1. どちらがOPSが高いのか（試合貢献度）")

print(f"パワーヒッター平均OPS: {df_power['OPS'].mean():.3f}")
print(f"アベレージヒッター平均OPS: {df_average['OPS'].mean():.3f}")
print(f"差: {abs(df_power['OPS'].mean() - df_average['OPS'].mean()):.3f}")
if p_value_ops < 0.05:
    print(f"\n結論: {ops_winner}の方が統計的に有意にOPSが高い")
else:
    print(f"\n結論: 統計的に有意な差なし（同程度の試合貢献度）")

print("2. どちらが年俸が高いのか（市場価値）")
print(f"パワーヒッター平均年俸: {df_power['salary'].mean():.0f}万円")
print(f"アベレージヒッター平均年俸: {df_average['salary'].mean():.0f}万円")
print(f"差: {abs(df_power['salary'].mean() - df_average['salary'].mean()):.0f}万円")
if p_value_salary < 0.05:
    print(f"\n結論: {salary_winner}の方が統計的に有意に年俸が高い")
else:
    print(f"\n結論: 統計的に有意な差なし（同程度の市場価値）")

print(f"\nコストパフォーマンス: {cp_winner}の方が優れている")
print(f"  パワーヒッター: {df_power['OPS_per_10M'].mean():.4f} OPS/1000万円")
print(f"  アベレージヒッター: {df_average['OPS_per_10M'].mean():.4f} OPS/1000万円")

print("3. 年齢や血液型との関係")

print("\n【年齢】")
print(f"パワーヒッター平均年齢: {df_power['age'].mean():.1f}歳")
print(f"アベレージヒッター平均年齢: {df_average['age'].mean():.1f}歳")
if p_value_age < 0.05:
    print("結論: 年齢に統計的に有意な差あり")
else:
    print("結論: 年齢に統計的に有意な差なし")

print(f"\n年齢とOPSの相関:")
print(f"  パワーヒッター: {corr_age_ops_power:.3f}")
print(f"  アベレージヒッター: {corr_age_ops_average:.3f}")

print(f"\n年齢と年俸の相関:")
print(f"  パワーヒッター: {corr_age_salary_power:.3f}")
print(f"  アベレージヒッター: {corr_age_salary_average:.3f}")

print("\n【血液型】")
if 'p_value_blood' in locals() and p_value_blood < 0.05:
    print("結論: 血液型の分布に統計的に有意な差あり")
    print("→ 打撃スタイルと血液型に関係がある可能性")
else:
    print("結論: 血液型の分布に統計的に有意な差なし")
    print("→ 打撃スタイルと血液型に明確な関係はない")


print("【最終結論】")


if p_value_ops >= 0.05 and p_value_salary >= 0.05:
    print("\n✓ OPSも年俸も統計的に有意な差なし")
    print(f"✓ コストパフォーマンスは{cp_winner}が優れている")
    print(f"\n→ チームにとっては【{cp_winner}】の方が経済的に有利")
elif p_value_ops < 0.05:
    print(f"\n✓ {ops_winner}の方がOPSが高い（試合貢献度が高い）")
    if p_value_salary < 0.05:
        print(f"✓ {salary_winner}の方が年俸が高い（市場価値が高い）")
    print(f"✓ コストパフォーマンスは{cp_winner}が優れている")
    print(f"\n→ 総合的には、チームの予算と戦略に応じて選択すべき")
else:
    print(f"\n✓ OPSに有意差なし、年俸は{salary_winner}が高い")
    print(f"✓ コストパフォーマンスは{cp_winner}が優れている")
    print(f"\n→ 同じ貢献度なら、{cp_winner}の方が経済的に有利")
